In [ ]:
# out of the box python modules
import os
import sqlite3
import datetime

# some
from src.data import make_dataset
from src.models import make_prediction

# Data wrangling stuff
import pandas as pd
import numpy as np
from scipy.optimize import minimize
from matplotlib import rc

rc('mathtext', default='regular')

%load_ext dotenv
%dotenv

DATABASE_PATH = os.environ['DATABASE_PATH']
MODEL_STORE_PATH = os.environ['MODEL_STORE']
REPORTS_STORE_PATH = os.environ['REPORTS_STORE_PATH']

import warnings
warnings.filterwarnings('ignore')

In [ ]:
def sharpe_ratio(weights, returns, cov_matrix, risk_free_rate=0.0):
    portfolio_return = np.sum(weights * returns)
    portfolio_volatility = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))
    return (portfolio_return - risk_free_rate) / portfolio_volatility

In [ ]:
class PriceHolder:
    def __init__(self, tickers: list) -> None:
        self.tickers = list(tickers)
        self.weights = np.ones(len(tickers)) / len(tickers)
        self.datasets = None
        self.df_prices = None
        pass

    def make_tickers_dataset(self, start_date: str, end_date: str, prediction_window: int = 10):
        df_prices_holder = []
        self.datasets = []
        for fibra in self.tickers:
            current_df = make_dataset.get_ticker_dataset(fibra, start_date, end_date, prediction_window)
            self.datasets.append(current_df)
            df_prices_holder.append(current_df.query("source == 'actual'").avg_price)

        self.df_prices = pd.concat(df_prices_holder, axis=1, ignore_index=True)
        return
    
    def set_evaluation_dates(self):
        self.evaluation_dates = [df.date.max().date() for df in self.datasets]
        return
    
    def get_evaluation_dates(self):
        return list(self.evaluation_dates)

    def get_df_prices(self):
        return self.df_prices.copy()

    def get_datasets_list(self):
        return [df.copy() for df in self.datasets]

    def compute_optimal_weights(self):
        expected_returns = []
        expected_risks = []
        for df in self.datasets:
            asseet_std = df.query("source == 'actual'").avg_price.diff().std()
            current_price = df.query("source == 'actual'").iloc[-1].avg_price
            predicted_price = df.query("source == 'prediction'").iloc[-1].avg_price
            period_dividends = df.query("source == 'prediction'").dividends.sum()
            future_price_diff = predicted_price - current_price
            expected_pnl = future_price_diff + period_dividends

            expected_returns.append(expected_pnl)
            expected_risks.append(asseet_std)
        
        cov_matrix = self.df_prices.pct_change().cov()
        constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
        bounds = tuple((0, 1) for _ in range(len(self.tickers)))

        optimized = minimize(lambda x: -sharpe_ratio(x, expected_returns, cov_matrix), 
                            self.weights, method='SLSQP', bounds=bounds, constraints=constraints)

        self.weights =  optimized.x
        return
    
    def get_weights(self):
        return [round(item, 4) for item in self.weights]

In [ ]:
fibras = ['FIBRAMQ12.MX', 'FIBRAPL14.MX', 'FMTY14.MX', 'FNOVA17.MX', 'DANHOS13.MX'] # , 'DANHOS13.MX'
# fibras = ['FIBRAMQ12.MX']

In [ ]:
prediction_window = 20
trailing_days = 450
study_start_date = datetime.datetime.strptime('2023-01-01', '%Y-%m-%d')
study_end_date = datetime.datetime.strptime('2024-01-01', '%Y-%m-%d')


end_date = study_start_date.date()
start_date = end_date - datetime.timedelta(days=trailing_days)


price_holder = PriceHolder(fibras)
price_holder.make_tickers_dataset(str(start_date), str(end_date), prediction_window=prediction_window)


innitial_investment = 10_000
accumulated_pnl = 0

In [ ]:
# Get initial values and number of shares
buy_date = study_start_date.date()
evaluation_date = min([df.date.max().date() for df in price_holder.datasets])
buy_prices = [df.get(df.date >= str(buy_date)).avg_price.values[0] for df in price_holder.datasets]
sell_prices = [df.get(df.date == str(evaluation_date)).avg_price.values[-1] for df in price_holder.datasets]

last_weights = price_holder.get_weights()
share_number = [round(innitial_investment * weight / price) for weight, price in zip(last_weights, buy_prices)]
reminder = sum([(innitial_investment * weight) - price * round(innitial_investment * weight / price)  for weight, price in zip(last_weights, buy_prices)])

# Optmize weights with forecasted prices
price_holder.compute_optimal_weights()

# Rebalance portfolio and compute values
sells = [min(shares * (new/last - 1), 0) for last, new, shares in zip(last_weights, price_holder.get_weights(), share_number)]
new_balance = -sum([number * sell_price for number, sell_price in zip(sells, sell_prices)]) + reminder
new_shares = [round(new_balance * weight / price) for weight, price in zip(price_holder.get_weights(), sell_prices)]
new_share_number = [last_number + sell_number + new_number for new_number, last_number, sell_number in zip(new_shares, share_number, sells)]

current_pnl = sum([share * price for share, price in zip(new_share_number, sell_prices)]) - innitial_investment

In [ ]:
while end_date <= study_end_date.date():
    end_date = evaluation_date
    price_holder.make_tickers_dataset(str(start_date), str(end_date), prediction_window=prediction_window)

    # Get initial values
    sell_prices = [df.get(df.date == str(evaluation_date)).avg_price.values[-1] for df in price_holder.datasets]
    evaluation_date = min([df.date.max().date() for df in price_holder.datasets])

    # Get initial/current number of shares
    last_weights = price_holder.get_weights()
    share_number = list(new_share_number)

    # Optmize weights with forecasted prices
    price_holder.compute_optimal_weights()

    # Rebalance portfolio and compute values
    sells = [np.nan_to_num(min(shares * (new/last - 1), 0), nan=0) for last, new, shares in zip(last_weights, price_holder.get_weights(), share_number)]
    new_balance = -sum([number * sell_price for number, sell_price in zip(sells, sell_prices)]) + reminder
    new_shares = [round(new_balance * weight / price) for weight, price in zip(price_holder.get_weights(), sell_prices)]
    new_share_number = [last_number + sell_number + new_number for new_number, last_number, sell_number in zip(new_shares, share_number, sells)]
    reminder = sum([(new_balance * weight) - (price * round(new_balance * weight / price))  for weight, price in zip(price_holder.get_weights(), sell_prices)])

    current_pnl = sum([share * price for share, price in zip(new_share_number, sell_prices)]) + reminder - innitial_investment
    
    print(str(evaluation_date), current_pnl)

In [ ]:
end_date = evaluation_date
price_holder.make_tickers_dataset(str(start_date), str(end_date), prediction_window=prediction_window)

In [ ]:
# Get initial values
sell_prices = [df.get(df.date == str(evaluation_date)).avg_price.values[-1] for df in price_holder.datasets]
evaluation_date = min([df.date.max().date() for df in price_holder.datasets])

# Get initial/current number of shares
last_weights = price_holder.get_weights()
share_number = list(new_share_number)

# Optmize weights with forecasted prices
price_holder.compute_optimal_weights()

# Rebalance portfolio and compute values
sells = [np.nan_to_num(min(shares * (new/last - 1), 0), nan=0) for last, new, shares in zip(last_weights, price_holder.get_weights(), share_number)]
new_balance = -sum([number * sell_price for number, sell_price in zip(sells, sell_prices)]) + reminder
new_shares = [round(new_balance * weight / price) for weight, price in zip(price_holder.get_weights(), sell_prices)]
new_share_number = [last_number + sell_number + new_number for new_number, last_number, sell_number in zip(new_shares, share_number, sells)]
reminder = sum([(new_balance * weight) - (price * round(new_balance * weight / price))  for weight, price in zip(price_holder.get_weights(), sell_prices)])

current_pnl = sum([share * price for share, price in zip(new_share_number, sell_prices)]) + reminder - innitial_investment

In [ ]:
end_date

In [ ]:
evaluation_date

In [ ]:
last_weights

In [ ]:
price_holder.get_weights()

In [ ]:
share_number

In [ ]:
new_share_number

In [ ]:
sell_prices

In [ ]:
sells

In [ ]:
new_balance

In [ ]:
current_pnl